In [138]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

! pip install mlxtend


# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

you can refer to this article to find out about apriori and association rule:
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here: 
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [139]:
# load the data set and show the first five transaction
url = "https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv"
df = pd.read_csv(url)
df.head()

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


Get the unique product that has been purchased

In [140]:
# Get the set of products that have been purchased
unique_products_set = set(df.iloc[:, 6].unique())

# Display the set of unique products
print(unique_products_set)

{nan, 'Bread', 'Diaper', 'Cheese', 'Eggs', 'Wine', 'Bagel', 'Milk', 'Pencil', 'Meat'}


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [141]:
one_hot_encoded = pd.get_dummies(df.iloc[:, 6], dummy_na=True)
# Display the dataframe with one-hot encoding (including NaN columns)
print("\nDataframe with one-hot encoding (including NaN columns):")
one_hot_encoded.head()


Dataframe with one-hot encoding (including NaN columns):


,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine,NaN
0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

In [142]:
# Drop NaN columns
df_encoded_cleaned = df_encoded.dropna(axis=1)

# OR

# Select all columns other than the first column
df_encoded_cleaned = df_encoded.iloc[:, 1:]

# Display the cleaned dataframe
print("\nCleaned Dataframe:")
df_encoded_cleaned.head()



Cleaned Dataframe:


,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine,NaN
0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,1


## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products. 
For this case study, we will min_support=0.2

In [143]:
frequent_itemsets = apriori(df_encoded_cleaned, min_support=0.2, use_colnames=True)
frequent_itemsets

/Users/marshalikorawung/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.869841,(nan)


Then, we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [144]:
association_rules_df = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)
association_rules_df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


Provide explanation about __antecedent support__, __consequent support__, __support__, __confidence__, __lift__, __leverage__ and __conviction__

1. **Support**: 
   support(A→C)=support(A ∪ C),range: [0,1]
   
   The support metric is defined for itemsets, not assocication rules. The table produced by the association rule mining algorithm contains three different support metrics: 'antecedent support', 'consequent support', and 'support'. 
   
   - Antecedent support computes the proportion of transactions that contain the antecedent A.

   - Consequent support computes the support for the itemset of the consequent C. 

   - Support metric then computes the support of the combined itemset A ∪ C.

   Typically, support is used to measure the abundance or frequency (often interpreted as significance or importance) of an itemset in a database. We refer to an itemset as a "frequent itemset" if you support is larger than a specified minimum-support threshold. Note that in general, due to the downward closure property, all subsets of a frequent itemset are also frequent.

2. **Confidence:**
   confidence(A→C)=support(A→C) / support(A),range: [0,1]

   The confidence of a rule A->C is the probability of seeing the consequent in a transaction given that it also contains the antecedent. Note that the metric is not symmetric or directed; for instance, the confidence for A->C is different than the confidence for C->A. The confidence is 1 (maximal) for a rule A->C if the consequent and antecedent always occur together.

3. **Lift:**
   lift(A→C)=confidence(A→C) / support(C),range: [0,∞]

   The lift metric is commonly used to measure how much more often the antecedent and consequent of a rule A->C occur together than we would expect if they were statistically independent. If A and C are independent, the Lift score will be exactly 1.

4. **Leverage:**
   levarage(A→C)= support(A→C) − support(A) × support(C),range: [−1,1]

   Leverage computes the difference between the observed frequency of A and C appearing together and the frequency that would be expected if A and C were independent. A leverage value of 0 indicates independence.

5. **Conviction:**
  conviction(A→C)=1 − support(C) / 1 − confidence(A→C),range: [0,∞]

   A high conviction value means that the consequent is highly depending on the antecedent. For instance, in the case of a perfect confidence score, the denominator becomes 0 (due to 1 - 1) for which the conviction score is defined as 'inf'. Similar to lift, if items are independent, the conviction is 1.

Reference : https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/ 